In [ ]:
index = 'shopping_demo_test'
loadType = 'text' #text / image / text_and_image
llmKeywords = 'No'
llmModel = 'anthropic.claude-3-haiku-20240307-v1:0'
language = 'english'

textEmbeddingEndpoint = 'bge-m3-xxxxxx-endpoint'
imageEmbeddingEndpoint = ''
textEmbeddingModelId = ''
imageEmbeddingModelId = ''

imageColoumName = 'ImageURL'
textColoumNameList = 'ProductTitle,Colour,Category,Gender,SubCategory'

local_file = "fashion.csv"

model_parameters = {"max_tokens_to_sample": 700,"temperature": 0}
prompt_template = """
you are a e-commerce website product manager, Your task is to extract the product's features base on product infomation, to make it easier for customers to search for the product, 
and the product's key features include: name, category, color, size, material and functionality, The feature text needs to be short, output as json format.

<product information>
{product_information}
</product information>

No need to preface, directly output the product’s key features as json format.

"""

In [ ]:
import sys
sys.path.append(r"../../lambda/lambda_layer_folder/python")

import re
import os
import json
import traceback
import urllib.parse
import boto3
from datetime import datetime
import time
from opensearch_search import add_products,get_opensearch_client
from embeddings import get_image_embedding_sagemaker,get_embedding_sagemaker,get_embedding_bedrock,get_embedding_bedrock_multimodal
from bedrock_model import invoke_model_local
import csv
from tqdm.notebook import trange 


product_info_list = []
product_embedding_list = []
image_embedding_list = []
metadatas = []
i = 0

csvfile=open(local_file,mode='r',encoding='utf-8')
reader = [each for each in csv.DictReader(csvfile, delimiter=',')]

now1 = datetime.now()
for i in trange(len(reader)):
    line = reader[i]
    metadata = {}
    image_url = ''
    product_info = ''
    keyFeatures = {}

    for key in line.keys():
        key_name = key.replace('\ufeff','').strip()
        value = line[key].strip()

        if len(key_name) > 0:
            metadata[key_name] = value
            if len(imageColoumName) > 0 and key_name == imageColoumName:
                image_url = value
            if len(textColoumNameList) > 0 and key_name in textColoumNameList:
                if llmKeywords == "Yes":
                    keyFeatures[key_name] = value
                else:
                    if len(product_info) == 0:
                        product_info = value
                    else:
                        product_info += (',' + value)

    if llmKeywords == "Yes":
        product_information = json.dumps(keyFeatures)
        print('product_information:',product_information)
        prompt = prompt_template.format(product_information=product_information)
        #print('prompt:',prompt)
        if True:
        # try:
            search_term = invoke_model_local(prompt,llmModel,model_parameters)
            metadata['SEARCH_TERM'] = search_term
            product_info = search_term
        # except:
        #     print("invoke model to get search term error")

    image_embedding = ''
    if loadType.find('image') >= 0 and len(image_url) > 0:
        try:
            if len(imageEmbeddingModelId) > 0:
                image_embedding = get_embedding_bedrock_multimodal(url=image_url,model_id=imageEmbeddingModelId)
            elif len(imageEmbeddingEndpoint) > 0:
                image_embedding = get_image_embedding_sagemaker(imageEmbeddingEndpoint,image_url)
        except:
            error_records.append(metadata)
            print("image embedding error")

    text_embedding = ''
    if loadType.find('text') >= 0 and len(product_info) > 0:
        try:
            if len(textEmbeddingModelId) > 0:
                text_embedding = get_embedding_bedrock(product_info,textEmbeddingModelId)
            elif len(textEmbeddingEndpoint) > 0:
                text_embedding = get_embedding_sagemaker(textEmbeddingEndpoint,product_info,language=language)
        except:
            error_records.append(metadata)
            print("text embedding error")

    if loadType.find('image') >= 0 and loadType.find('text') >= 0:
        if len(image_embedding) == 0 or len(text_embedding) == 0:
            continue
    elif loadType.find('image') >= 0 and len(image_embedding) == 0:
        continue
    elif loadType.find('text') >= 0 and len(text_embedding) == 0:
        continue
    
    metadatas.append(metadata)
    if len(product_info) > 0:
        product_info_list.append(product_info)
    if len(text_embedding) > 0:
        product_embedding_list.append(text_embedding)
    if len(image_embedding) > 0:
        image_embedding_list.append(image_embedding)
            
    if i% 100 == 0:
        # print('product_info_list len:',len(product_info_list))
        # print('metadatas len:',len(metadatas))
        # print('product_embedding_list len:',len(product_embedding_list))

        if len(image_embedding_list) > 0 or len(product_embedding_list) > 0:
            add_products(index,product_info_list,product_embedding_list,metadatas,image_embedding_list)
            product_info_list = []
            product_embedding_list = []
            image_embedding_list = []
            metadatas = []

        print('finish add products to opensearch,index is:' + index)
        
print('product number:',len(product_info_list))
add_products(index,product_info_list,product_embedding_list,image_embedding_list,metadatas)
print('finish add products to opensearch,index is:' + index)

now2 = datetime.now()
print("Total File import takes time:",now2-now1)